In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,row_number,max,min
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

auto_repair = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "auto_repair") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

auto_repair.show()

24/09/26 12:40:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/26 12:40:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+------+----+-----------+---------+-------+
|client|auto|repair_date|indicator|  value|
+------+----+-----------+---------+-------+
|    c1|  a1|       2022|    level|   good|
|    c1|  a1|       2022| velocity|     90|
|    c1|  a1|       2023|    level|regular|
|    c1|  a1|       2023| velocity|     80|
|    c1|  a1|       2024|    level|  wrong|
|    c1|  a1|       2024| velocity|     70|
|    c2|  a1|       2022|    level|   good|
|    c2|  a1|       2022| velocity|     90|
|    c2|  a1|       2023|    level|  wrong|
|    c2|  a1|       2023| velocity|     50|
|    c2|  a2|       2024|    level|   good|
|    c2|  a2|       2024| velocity|     80|
+------+----+-----------+---------+-------+



In [7]:
auto_repair.createOrReplaceTempView("auto_repair")
spark.sql('''
with cte_1 as (
select
    *,
    lead(value) over (partition by repair_date order by (select null)) as velocity
from
    auto_repair),
cte_2 as (
select
    value,
    velocity,
    count(*) as cnt
from
    cte_1
where
    velocity REGEXP '^[0-9]+$'
group by
    value, velocity),
cte_3 as (
select
    *,
    case
        when value = 'good' and velocity = 90 then cnt else 0
    end as good_1,
    case
        when value = 'good' and velocity = 80 then cnt else 0
    end as good_2,
    case
        when value = 'good' and velocity = 70 then cnt else 0
    end as good_3,
    case
        when value = 'good' and velocity = 50 then cnt else 0
    end as good_4,
    case
        when value = 'wrong' and velocity = 50 then cnt else 0
    end as wrong_1,
    case
        when value = 'wrong' and velocity = 70 then cnt else 0
    end as wrong_2,
    case
        when value = 'regular' and velocity = 80 then cnt else 0
    end as regular
from
    cte_2)
select
    velocity,
    sum(good_1+good_2+good_3+good_4) as good,
    sum(wrong_1+wrong_2) as wrong,
    sum(regular) as regular
from
    cte_3
group by
    velocity
order by
    velocity
''').show()

+--------+----+-----+-------+
|velocity|good|wrong|regular|
+--------+----+-----+-------+
|      50|   0|    1|      0|
|      70|   0|    1|      0|
|      80|   1|    0|      1|
|      90|   2|    0|      0|
+--------+----+-----+-------+

